In [1]:
import numpy as np

#For plotting
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

#For colorblind plots
import seaborn
colors = list(seaborn.color_palette('colorblind').as_hex())

#For TeX plots
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

#For the dilogarithm
from scipy.special import spence

#For increased precision
import mpmath as mp
mp.mp.dps = 30

In [2]:
def Li2(x):
    # spence doesn't work with mp, so have to 
    # convert back to numpy
    x = np.complex128(x)
    return spence(1-x)

def DiLog(u, e):
    # np.where doesn't work with mp, so have to
    # convert back to numpy
    e = np.complex128(e)
    return np.where(e < 0, Li2(u), -0.5*mp.log(1/(1-u))**2-Li2(u/(u-1)))

def KallenL(x, y, z):
    return x**2 + y**2 + z**2 - 2*x*y - 2*x*z - 2*y*z

def DiLog_fn(x, z, e):
    argm = 2*e/(-1+x**2 + z**2 - mp.sqrt(KallenL(1, x**2, z**2)))
    argp = 2*e/(-1+x**2 + z**2 + mp.sqrt(KallenL(1, x**2, z**2)))
    return DiLog(argm, e) + DiLog(argp, -e)

#Special case of the ScalarC0 function from PackageX
def C0(x,y,z):  
    Dxy = DiLog_fn(x, z, -1+z**2) - DiLog_fn(x, z, -1+x**2+z**2) 
    Dyx = DiLog_fn(y, z, -1+z**2) - DiLog_fn(y, z, -1+y**2+z**2)
    
    return (Dxy - Dyx)/(x**2 - y**2)

def B(x, y):
    arg = (1-x**2+y**2 + mp.sqrt(KallenL(1, x**2, y**2)))/(2*y)
    return mp.sqrt(KallenL(1, x**2, y**2))*mp.log(arg)/x**2

In [3]:
def fBik(x, y, z):
    return ((2*z+y)*x**2 - (2*x + y)*(z**2 + 2*x*z - 1))/(x*(x**2 - y**2))

def fBjk(x, y, z):  
    return fBik(y, x, z)

def fLog(x, y, z): 
    return -(((x+y)*(1-z**2)-x*y*z)**2 - 3*(x*y*z)**2)/(x**3*y**3)

def fConst(x, y, z):  
    return -(1+x*y-z**2)/(x*y)

def fC0(x, y, z): 
    return -2*(x+y+z)*z

def f(x, y, z):
    if hasattr(x, '__iter__'):
        return np.array([f(x_, y_, z_) for x_,y_,z_ in zip(x,y,z)])
    
    #increased precision
    x = mp.mpc(x)
    y = mp.mpc(y)
    z = mp.mpc(z)
    
    f_val = fBik(x, y, z)*B(x, z)
    f_val+= fBjk(x, y, z)*B(y, z)
    f_val+= fLog(x, y, z)*mp.log(z)
    f_val+= fConst(x, y, z)
    f_val+= fC0(x, y, z)*C0(x, y, z)
    
    return np.complex128(f_val)

def f_plus(x, y, z):
    return (f(x, y, z) + f(-x, -y, z))/2

def f_minus(x, y, z):
    return (f(x, y, z) - f(-x, -y, z))/2

In [4]:
#approximations
def f_minus_iji(xi, xj, xk):
    f_val = -2 + (xi - 3)*xi/(xi-1)*np.log(xi)
    f_val-= 2*np.sqrt(xi*(xi-4))*np.log((xi + np.sqrt(xi*(xi-4)))/(2*np.sqrt(xi)))
    f_val-= np.log((xi + np.sqrt(xi*(xi-4)))/(2*xi))**2
    f_val-= 2*Li2(1-xi)
    f_val-= 2*Li2((xi-np.sqrt(xi*(xi-4)))/(2*xi))
    f_val+= 2*Li2((2-xi-np.sqrt(xi*(xi-4)))/2)
    return f_val

def f_plus_iji(xi, xj, xk):
    f_val = 2*xi - 3 - (xi - 3)*xi*np.log(xi)
    f_val+= 2*(xi-1)*np.sqrt(xi*(xi-4))*np.log((xi + np.sqrt(xi*(xi-4)))/(2*np.sqrt(xi)))
    f_val-= np.log((xi + np.sqrt(xi*(xi-4)))/(2*xi))**2
    f_val-= 2*Li2(1-xi)
    f_val-= 2*Li2((xi-np.sqrt(xi*(xi-4)))/(2*xi))
    f_val+= 2*Li2((2-xi-np.sqrt(xi*(xi-4)))/2)
    return f_val

def f_minus_small_k(xi, xj, xk):
    f_val = 1 + (np.log((xi-1)*xk/xi) + xi - 1)*np.log((xi-1)/xi) + Li2(1/xi)
    return np.conjugate(-2*np.sqrt(xi)/np.sqrt(xk)*f_val)

def f_plus_small_k(xi, xj, xk):
    return np.conjugate((-1 + 2*xi + 2*(xi-1)*xi*np.log((xi-1)/xi)))

def f_minus_large_k(xi, xj, xk):
    return np.sqrt(xk/xi)*(-(3*xk-1)/(xk-1)**2 + 2*xk**2/(xk-1)**3 * np.log(xk))

def f_plus_large_k(xi, xj, xk):
    f_val = (2*xk**2 + 5*xk - 1)/(6*(xk-1)**3) - xk**2/(xk-1)**4 * np.log(xk)
    return xk*(1/xi+1/np.sqrt(xi*xj)) * f_val
        
def f_large_m(x, y, z):
    return -2*z*(x+y+z)*(1+2*np.log(z))+((x+y)**2 - 3*(x+y)*z+6*z**2+12*z**2*np.log(z))

def f_minus_large_m(x, y, z):
    return -(x+y)*z*(3 + 4*np.log(z))

def f_plus_large_m(x, y, z):
    return (x+y)**2/3

In [16]:
leptons = ['e', '\\mu', '\\tau']
processes = [(1, 0, 0), (1, 0, 1), (1, 0, 2),
             (2, 0, 0), (2, 0, 1), (2, 0, 2),
             (2, 1, 0), (2, 1, 1), (2, 1, 2)]

def process_label(process):
    i, j, k = process
    label = f'{leptons[i]}\\stackrel{{{leptons[k]}}}{{\\longrightarrow}}{leptons[j]}'
    return '$' + label + '$'

#Check over a large range of masses
m = np.logspace(-4,4,1000, dtype=np.complex128)
me = 5.11e-4
mm = 0.106
mt = 1.77
ml = np.array([me, mm, mt])
u = [_/m for _ in ml]
x = [(m/_)**2 for _ in ml]

def f_plus_approx(process):
    i, j, k = process
    if i < j:
        return f_plus_approx((j, i, k))
    if i == j:
        #should be F2
        pass
    if i == k:
        return f_plus_iji(x[i], x[j], x[k])
    if k < i:
        return f_plus_small_k(x[i], x[j], x[k])
    if k > i:
        return f_plus_large_k(x[i], x[j], x[k])
    
def f_minus_approx(process):
    i, j, k = process
    if i < j:
        return f_plus_approx((j, i, k))
    if i == j:
        #should be F2
        pass
    if i == k:
        return f_plus_iji(x[i], x[j], x[k])
    if k < i:
        return f_plus_small_k(x[i], x[j], x[k])
    if k > i:
        return f_plus_large_k(x[i], x[j], x[k])
    
fp_approx = {}
fm_approx = {}

fp = {}
fm = {}

fp3 = {}
fm3 = {}

labels = {}
for process in processes:
    i, j, k = process
    fp_approx[process] = f_plus_approx(process)
    fm_approx[process] = f_minus_approx(process)
    
    fp[process] = np.where(x[i] < 1e6, f_plus(u[i], u[j], u[k]), f_plus_large_m(u[i], u[j], u[k]))
    fm[process] = np.where(x[i] < 1e6, f_minus(u[i], u[j], u[k]), f_minus_large_m(u[i], u[j], u[k]))
    
    fp3[process] = np.where(x[i] < 1e6, f_plus(u[i], -u[j], u[k]), f_plus_large_m(u[i], -u[j], u[k]))
    fm3[process] = np.where(x[i] < 1e6, f_minus(u[i], -u[j], u[k]), f_minus_large_m(u[i], -u[j], u[k]))  
    
    labels[process] = process_label(process)

In [17]:
def error(x, y):
    return np.abs((y-x))
def rel_error(x, y):
    return np.abs((y-x)/x)

In [ ]:
# Create a figure and use GridSpec to control layout precisely
fig = plt.figure(figsize=(10, 8))
gs = GridSpec(3, 1, height_ratios=[2, 0, 1], hspace = 0)  # hspace=0 removes gap

# Top plot: Exact functions and approximation
ax1 = fig.add_subplot(gs[0])

relevant_processes = [(1, 0, 1), (2, 0, 2), (2, 1, 2)]

#Exact results
for idx, process in enumerate(relevant_processes):
    i, j, k = process
    ax1.plot(x[i], fp(process), color = colors[idx], label = labels[process])
    ax1.plot(x[i], fp_approx(process), color = 'black', linestyle = 'dotted')

    

    
ax1.plot(xt, fp_tmt, color=colors[0], linewidth = 6)
ax1.plot(xt, fp_tet, color=colors[1], linewidth = 4)
ax1.plot(xm, fp_mem, color=colors[2], linewidth = 1)

#Approximation
ax1.plot(xt, f_plus_iji(xt), color='black', linestyle="dotted")

#Dummy plots for legend
ax1.plot([1000], label="$\\mu \\stackrel{\\mu}{\\longrightarrow} e\\gamma$", color=colors[2], linewidth = 2)
ax1.plot([1000], label="$\\tau \\stackrel{\\tau}{\\longrightarrow} e\\gamma$", color=colors[1], linewidth = 2)
ax1.plot([1000], label="$\\tau\\stackrel{\\tau}{\\longrightarrow}\\mu\\gamma$", color=colors[0], linewidth = 2)

ax1.plot([1000], label="{\\textrm{approx.}}", color='black', linestyle="dotted", linewidth = 2)

#Legend
ax1.legend(fancybox=False, facecolor = 'white', edgecolor = 'black', fontsize = 12, framealpha = 1)

#Axis specifications
ax1.grid()

ax1.set_ylabel("$f_+(\\mu_i, \\mu_j, \\mu_i)$", fontsize = 14)
ax1.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax1.set_ylim(0, 0.4)

ax1.set_xscale('log')
ax1.set_xlim(1e-4,1e4)

# Bottom plot: Relative error
ax2 = fig.add_subplot(gs[2], sharex=ax1)

#Relative error
ax2.plot(xt, rel_error(fp_tmt, f_plus_iji(xt)), color=colors[0])
ax2.plot(xt, rel_error(fp_tet, f_plus_iji(xt)), color=colors[1])
ax2.plot(xm, rel_error(fp_mem, f_plus_iji(xm)), color=colors[2])

#Axis specifications
ax2.grid()

ax2.set_xlabel("$x_i=m_\\varphi^2/m_i^2$", fontsize = 14)

ax2.set_ylabel("$\\textrm{rel. errors}$", fontsize = 14)
ax2.set_yscale('log')
ax2.set_ylim(1e-4,5e-1)

# Remove extra tick labels from the shared x-axis of the top plot
ax1.tick_params(which = 'both', direction = 'in', labelbottom=False)
ax2.tick_params(which = 'both', direction = 'in')

plt.show()